##  Oil Blending

Sunco Oil manufactures three types of gasoline (gas-1, gas-2, gas-3). Each type of gasoline is produced by blending three types of crude oil (crude-1, crude-2, crude-3). The details of this process are as follows.

- The sales price of each type of gasoline is given in the table below:

|Gasoline | Sales Price ($/Barrel) | 
|---------|------------------------|
|gas-1 | 70 |
|gas-2 | 60 |
|gas-3 | 50 | 

- The purchase price of crude oil is given in the table below:

|Crude Oil | Purchase Price ($/Barrel) |
|-----------|---------------------------|
| crude-1 | 45 |
| crude-2 | 35 |
| crude-3 | 25 |

- Sunco can purchase up to 5000 barrels of each type of crude oil daily.
- It costs $4 to transform one barrel of oil into one barrel of gasoline and Sunco's refinery can produce up to 14000 barrels of gasoline daily.
- The three types of crude oil differ in their octane levels and sulfur content, as detailed in the following table:

|Crude Oil | Octane Level | Sulfur Content (%) |
|----|-----|---|
| crude-1 | 12 | 0.5 |
| crude-2 | 6 | 2.0 |
| crude-3 | 8 | 3.0 |

- In a mixture of different types of crude oil, the octane levels and sulfur content are linearly blended.

- The mixture of gasoline must have an average level of
    - Octane of **at least** 10 for gas-1, 8 for gas-2, and 6 for gas-3.
    - Sulfur of **at most** 1% for gas-1, 2% for gas-2, and 1% for gas-3.

- Sunco is contractually obligated to meet (at least) the following demand:

|Gasoline | Demand (Barrels per day)|
|---------|---------|
| gas-1 | 3000 |
| gas-2 | 2000 |
| gas-3 | 1000 |

**Goal:** Plan Sunco's production to maximize its daily profit (revenue minus cost).

To solve this problem, we use decision variables $x_{ij}$ which represent the number of barrels of crude oil crude-$i$ used to produce gasoline gas-$j$.

We organize the data in terms of the type of crude oil and the type of gasoline as follows.

In [2]:
#Gas and Crude Types
gastype = [:gas_1, :gas_2, :gas_3]
crudetype = [:crude_1, :crude_2, :crude_3]

#Dictionary for sales price of each gas type
saleprice = Dict(zip(gastype, [70, 60, 50]))
#Dictionary for purchase price of each crude type
purchaseprice = Dict(zip(crudetype, [45, 35, 25]))

#Dictionary for octane level of each crude oil type
octanecrude = Dict(zip(crudetype, [12,6,8]))
#Dictionary for sulfur content of each crude oil type (in %)
sulfurcrude = Dict(zip(crudetype, [0.005,0.02,0.03]))

#Dictionary for minimum average octane level in each type of gasoline
octanegas = Dict(zip(gastype, [10, 8, 6]))
#Dictionary for maximum average sulfur level in each type of gasoline
sulfurgas = Dict(zip(gastype, [0.01, 0.02, 0.01]))

#Dictionary for demand of each type of gasoline
demand = Dict(zip(gastype, [3000, 2000, 1000]))

#Cost to transform one barrel of crude into 1 of gasoline
transformcost = 4
#Maximum purchasable barrels per day (for each type)
maxpurchase = 5000
#Max production per day (in barrels of gasoline)
maxproduce = 14000;

Now, we can create our model.

In [4]:
using JuMP, HiGHS

m = Model(HiGHS.Optimizer)

@variable(m, x[crudetype, gastype] >= 0)

#Maximum amount purchased for each crude
@constraint(m, mpurconstraint[i in crudetype], sum(x[i,j] for j in gastype) <= maxpurchase)
#Maximum amount produced per day
@constraint(m, mprodconstraint, sum(sum(x[i,j] for j in gastype) for i in crudetype) <= maxproduce)

#Meet demand obligations
@constraint(m, meetdemand[j in gastype], sum(x[i,j] for i in crudetype) >= demand[j])

#Minimum average octane constraint
@constraint(m, minoctane[j in gastype], sum(octanecrude[i]*x[i,j] for i in crudetype)
                                        >= sum(octanegas[j]*x[i,j] for i in crudetype))
#Maximum average sulfur constraint
@constraint(m, maxsulfur[j in gastype], sum(sulfurcrude[i]*x[i,j] for i in crudetype)
                                        <= sum(sulfurgas[j]*x[i,j] for i in crudetype))

#Objective is revenue (sales) minus cost (purchase and conversion)
@objective(m, Max, sum(saleprice[j]*sum(x[i,j] for i in crudetype) for j in gastype)
    - sum((purchaseprice[i]+transformcost)*sum(x[i,j] for j in gastype) for i in crudetype))

print(m)

In [6]:
status = optimize!(m)
blend = value.(x)
objval = objective_value(m)

println(status)
println(blend)
println(objval)

Solving LP without presolve or with basis
Model   status      : Optimal
Objective value     :  2.8850000000e+05
HiGHS run time      :          0.00
nothing
2-dimensional DenseAxisArray{Float64,2,...} with index sets:
    Dimension 1, [:crude_1, :crude_2, :crude_3]
    Dimension 2, [:gas_1, :gas_2, :gas_3]
And data, a 3×3 Matrix{Float64}:
 2000.0  2200.0  800.0
 1000.0  4000.0    0.0
   -0.0  3300.0  200.0
288500.0
